<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/262_MissionOrchestratorAgent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 0: Planning Document - Mission Orchestrator Agent

**Date:** 2025-12-11  
**Status:** Planning Complete  
**Purpose:** Deep analysis and architecture design before coding begins

---

## 1. Deep Data Analysis

### 1.1 Data Structure Inventory

#### **business_missions.json**
- **Structure:** Array of mission objects
- **Fields:**
  - `mission_id` (str): Unique identifier (M001, M002, M003)
  - `mission_name` (str): Human-readable name
  - `description` (str): Mission objective description
- **Assumptions:**
  - Mission IDs are unique and stable
  - Descriptions provide context for task decomposition
- **Edge Cases:**
  - Empty missions array (should error gracefully)
  - Missing mission_id (should error)
  - Duplicate mission_ids (should detect and error)

#### **decomposed_mission_tasks.json**
- **Structure:** Array of mission objects, each containing tasks array
- **Fields per task:**
  - `task_id` (str): Unique identifier (T1-T8)
  - `task` (str): Task description
  - `order` (int): Execution order (1, 2, 3...)
  - `depends_on` (List[str]): Array of task_ids that must complete first
  - `estimated_duration_minutes` (int): Time estimate for KPI tracking
  - `requires_human_approval` (bool): HITL flag
- **Assumptions:**
  - Tasks are ordered sequentially within a mission
  - `depends_on` creates a DAG (no circular dependencies)
  - First task in sequence has empty `depends_on` array
  - Duration estimates are reasonable (2-10 minutes)
- **Edge Cases:**
  - Circular dependencies (T1 depends on T2, T2 depends on T1) - should detect
  - Missing dependencies (T2 depends on T99 that doesn't exist) - should error
  - Tasks with same order number - should handle or error
  - Empty tasks array for a mission - should handle gracefully

#### **specialized_agents.json**
- **Structure:** Array of agent objects
- **Fields:**
  - `agent_id` (str): Unique identifier (A1-A5)
  - `name` (str): Human-readable name
  - `description` (str): Agent capability description
- **Assumptions:**
  - Agent IDs are unique and stable
  - Descriptions indicate what each agent can do
- **Edge Cases:**
  - Missing agent_id referenced in capabilities matrix
  - Duplicate agent_ids

#### **agent_capabilities_matrix.json**
- **Structure:** Array of task-to-agent mappings
- **Fields:**
  - `task_id` (str): Task identifier
  - `capable_agents` (List[str]): Array of agent_ids that can perform this task
- **Assumptions:**
  - Each task has at least one capable agent
  - Agent IDs match those in specialized_agents.json
- **Edge Cases:**
  - Task with no capable agents (should error)
  - Agent ID in capable_agents that doesn't exist (should error)
  - Task ID that doesn't exist in decomposed_mission_tasks.json (should error)

#### **mission_kpis.json**
- **Structure:** Array of mission KPI objects
- **Fields:**
  - `mission_id` (str): Mission identifier
  - `kpis` (Dict): KPI definitions (varies by mission)
- **KPI Examples:**
  - M001: `target_onboarding_time_days`, `baseline_onboarding_time_days`, `max_steps`
  - M002: `target_pipeline_days`, `baseline_pipeline_days`, `min_touchpoints`
  - M003: `target_resolution_time_hours`, `baseline_resolution_time_hours`, `csat_threshold`
- **Assumptions:**
  - Each mission has corresponding KPIs
  - Baseline values enable ROI calculation
  - Target values are measurable
- **Edge Cases:**
  - Mission without KPIs (should use defaults or error)
  - KPI values that are impossible (target > baseline when should be <)
  - Missing baseline values (should error for ROI calculation)

### 1.2 Data Relationships

```
Mission (M001)
  ├── Has Tasks (T1, T2, T3)
  │     ├── Each Task has Order, Dependencies, Duration, HITL flag
  │     └── Each Task maps to Agents via Capabilities Matrix
  └── Has KPIs (target, baseline, thresholds)
```

**Key Relationships:**
- Mission → Tasks (1:many)
- Task → Agents (many:many via capabilities matrix)
- Mission → KPIs (1:1)
- Task → Task (dependency graph via `depends_on`)

### 1.3 Data Loading Utilities Plan

**Files to create:**
- `utilities/data_loading.py`

**Functions needed:**
1. `load_business_missions()` → List[Dict]
2. `load_mission_tasks(mission_id)` → List[Dict] (tasks for specific mission)
3. `load_specialized_agents()` → List[Dict]
4. `load_agent_capabilities_matrix()` → List[Dict]
5. `load_mission_kpis(mission_id)` → Dict (KPIs for specific mission)
6. `get_mission_by_id(mission_id)` → Dict
7. `get_task_by_id(task_id)` → Dict
8. `get_agent_by_id(agent_id)` → Dict
9. `get_capable_agents_for_task(task_id)` → List[str] (agent_ids)
10. `validate_data_consistency()` → bool (validates all relationships)

---

## 2. Decision Rule Analysis

### 2.1 Mission Selection Rules

**Rule 1: Mission Selection**
- **Input:** `mission_id` (from user/input)
- **Logic:** Load mission, validate it exists
- **Output:** Mission object with tasks and KPIs
- **Dependencies:** None (entry point)

### 2.2 Task Decomposition Rules

**Rule 2: Task Ordering**
- **Input:** Mission tasks array
- **Logic:** Sort tasks by `order` field
- **Output:** Ordered task list
- **Dependencies:** Rule 1

**Rule 3: Dependency Resolution**
- **Input:** Ordered task list
- **Logic:**
  - For each task, check if dependencies are complete
  - Only execute tasks whose dependencies are satisfied
  - Detect circular dependencies (error)
- **Output:** Ready-to-execute task queue
- **Dependencies:** Rule 2

### 2.3 Agent Assignment Rules

**Rule 4: Agent Selection**
- **Input:** Task ID
- **Logic:**
  - Look up capable agents from capabilities matrix
  - If multiple agents, select first one (MVP: simple selection)
  - Validate agent exists
- **Output:** Selected agent_id
- **Dependencies:** Rule 3

### 2.4 Task Execution Rules

**Rule 5: Task Execution**
- **Input:** Task, Agent
- **Logic:**
  - Execute task via agent (MVP: mock execution)
  - Track start time, end time
  - Record result/status
- **Output:** Task execution result
- **Dependencies:** Rule 4

**Rule 6: Human Approval Check**
- **Input:** Task with `requires_human_approval: true`
- **Logic:**
  - Pause workflow
  - Request human approval
  - Wait for approval/rejection
  - Continue or abort based on decision
- **Output:** Approval status
- **Dependencies:** Rule 5

### 2.5 Progress Tracking Rules

**Rule 7: Progress Calculation**
- **Input:** Completed tasks, total tasks
- **Logic:** `progress = (completed / total) * 100`
- **Output:** Progress percentage
- **Dependencies:** Rule 5

**Rule 8: KPI Tracking**
- **Input:** Task durations, mission KPIs
- **Logic:**
  - Sum task durations → actual time
  - Compare to target KPI
  - Calculate improvement: `(baseline - actual) / baseline * 100`
- **Output:** KPI metrics
- **Dependencies:** Rule 5

### 2.6 Mission Completion Rules

**Rule 9: Mission Completion Check**
- **Input:** All tasks status
- **Logic:** All tasks completed AND all approvals granted
- **Output:** Mission status (in_progress, completed, failed)
- **Dependencies:** Rule 5, Rule 6

### 2.7 Rule Dependency Diagram

```
Rule 1: Mission Selection
  ↓
Rule 2: Task Ordering
  ↓
Rule 3: Dependency Resolution
  ↓
Rule 4: Agent Selection
  ↓
Rule 5: Task Execution
  ↓
Rule 6: Human Approval (if needed)
  ↓
Rule 7: Progress Calculation
Rule 8: KPI Tracking
  ↓
Rule 9: Mission Completion Check
```

---

## 3. State Schema Design

### 3.1 Complete State Schema

See `config.py` for `MissionOrchestratorState` TypedDict definition.

**Key State Fields:**

**Input:**
- `mission_id` (str): Mission to execute

**Goal & Planning:**
- `goal` (Dict): Mission goal definition
- `plan` (List[Dict]): Execution plan with tasks

**Mission Data:**
- `mission` (Dict): Loaded mission object
- `mission_tasks` (List[Dict]): All tasks for mission
- `mission_kpis` (Dict): KPIs for mission

**Agent Data:**
- `specialized_agents` (List[Dict]): All available agents
- `capabilities_matrix` (List[Dict]): Task-to-agent mappings

**Task Execution:**
- `task_queue` (List[Dict]): Tasks ready to execute (dependencies satisfied)
- `executed_tasks` (List[Dict]): Completed tasks with results
- `current_task` (Optional[Dict]): Currently executing task
- `task_results` (Dict[str, Dict]): task_id → execution result

**Progress Tracking:**
- `progress_percentage` (float): 0-100
- `tasks_completed` (int): Count of completed tasks
- `tasks_total` (int): Total tasks in mission
- `elapsed_time_minutes` (float): Time since mission start
- `estimated_remaining_minutes` (float): Estimated time to completion

**KPI Metrics:**
- `kpi_metrics` (Dict): Current KPI values
- `kpi_status` (Dict): KPI achievement status (on_track, at_risk, exceeded)

**HITL (Human-in-the-Loop):**
- `pending_approvals` (List[Dict]): Tasks awaiting human approval
- `approval_history` (List[Dict]): Approval decisions made

**Mission Status:**
- `mission_status` (str): "not_started", "in_progress", "awaiting_approval", "completed", "failed"
- `completion_reason` (Optional[str]): Why mission completed/failed

**Output:**
- `mission_report` (str): Final markdown report
- `report_file_path` (Optional[str]): Path to saved report

**Metadata:**
- `errors` (List[str]): Any errors encountered
- `processing_time` (Optional[float]): Total processing time

### 3.2 State Enrichment Flow

```
Initial State:
  {mission_id: "M001"}

After Goal Node:
  {mission_id, goal}

After Planning Node:
  {mission_id, goal, plan}

After Data Loading Node:
  {mission_id, goal, plan, mission, mission_tasks, mission_kpis, specialized_agents, capabilities_matrix}

After Task Execution (iterative):
  {..., task_queue, executed_tasks, current_task, task_results, progress_percentage, ...}

After Progress Tracking:
  {..., kpi_metrics, kpi_status, ...}

After Mission Completion:
  {..., mission_status: "completed", mission_report, report_file_path}
```

---

## 4. Architecture Planning

### 4.1 Node Structure

**Nodes (in execution order):**

1. **goal_node**
   - **Responsibility:** Define mission goal
   - **Input:** `mission_id`
   - **Output:** `goal` (Dict)
   - **Dependencies:** None

2. **planning_node**
   - **Responsibility:** Create execution plan
   - **Input:** `goal`, `mission_id`
   - **Output:** `plan` (List[Dict])
   - **Dependencies:** goal_node

3. **data_loading_node**
   - **Responsibility:** Load all mission and agent data
   - **Input:** `mission_id`
   - **Output:** `mission`, `mission_tasks`, `mission_kpis`, `specialized_agents`, `capabilities_matrix`
   - **Dependencies:** planning_node

4. **task_ordering_node**
   - **Responsibility:** Order tasks and resolve dependencies
   - **Input:** `mission_tasks`
   - **Output:** `task_queue` (ordered, dependencies resolved)
   - **Dependencies:** data_loading_node

5. **task_execution_node** (iterative/conditional)
   - **Responsibility:** Execute next task in queue
   - **Input:** `task_queue`, `current_task`, `specialized_agents`, `capabilities_matrix`
   - **Output:** `executed_tasks`, `task_results`, `current_task`
   - **Dependencies:** task_ordering_node
   - **Special:** This node may loop until all tasks complete

6. **approval_check_node** (conditional)
   - **Responsibility:** Check if task requires approval, handle HITL
   - **Input:** `current_task`, `task_results`
   - **Output:** `pending_approvals`, `approval_history`
   - **Dependencies:** task_execution_node
   - **Special:** Only executes if `requires_human_approval: true`

7. **progress_tracking_node**
   - **Responsibility:** Calculate progress and KPI metrics
   - **Input:** `executed_tasks`, `mission_kpis`, `elapsed_time_minutes`
   - **Output:** `progress_percentage`, `kpi_metrics`, `kpi_status`
   - **Dependencies:** task_execution_node

8. **completion_check_node** (conditional)
   - **Responsibility:** Check if mission is complete
   - **Input:** `executed_tasks`, `tasks_total`, `pending_approvals`
   - **Output:** `mission_status`
   - **Dependencies:** progress_tracking_node
   - **Special:** Routes to report_generation_node if complete, or back to task_execution_node if not

9. **report_generation_node**
   - **Responsibility:** Generate final mission report
   - **Input:** All state fields
   - **Output:** `mission_report`, `report_file_path`
   - **Dependencies:** completion_check_node

### 4.2 Utility Structure

**Utilities to create:**

**`utilities/data_loading.py`**
- `load_business_missions()`
- `load_mission_tasks(mission_id)`
- `load_specialized_agents()`
- `load_agent_capabilities_matrix()`
- `load_mission_kpis(mission_id)`
- `get_mission_by_id(mission_id)`
- `get_task_by_id(task_id)`
- `get_agent_by_id(agent_id)`
- `get_capable_agents_for_task(task_id)`
- `validate_data_consistency()`

**`utilities/task_ordering.py`**
- `order_tasks_by_dependency(tasks)` → List[Dict]
- `resolve_task_dependencies(tasks)` → List[Dict] (ready to execute)
- `detect_circular_dependencies(tasks)` → bool
- `get_ready_tasks(tasks, completed_task_ids)` → List[Dict]

**`utilities/agent_selection.py`**
- `select_agent_for_task(task_id, capabilities_matrix, agents)` → Dict (agent)
- `get_capable_agents(task_id, capabilities_matrix)` → List[Dict]

**`utilities/task_execution.py`** (MVP: Mock execution)
- `execute_task(task, agent)` → Dict (result)
- `mock_agent_execution(agent_id, task_description)` → Dict (mock result)

**`utilities/progress_tracking.py`**
- `calculate_progress(completed_count, total_count)` → float
- `calculate_elapsed_time(start_time)` → float
- `estimate_remaining_time(completed_tasks, remaining_tasks)` → float

**`utilities/kpi_tracking.py`**
- `calculate_kpi_metrics(executed_tasks, mission_kpis)` → Dict
- `assess_kpi_status(kpi_metrics, mission_kpis)` → Dict
- `calculate_roi_improvement(baseline, actual)` → float

**`utilities/report_generation.py`**
- `generate_mission_report(state)` → str (markdown)
- `save_report(report_content, mission_id)` → str (filepath)

### 4.3 Workflow Structure

**Linear Flow (MVP):**
```
goal_node
  → planning_node
  → data_loading_node
  → task_ordering_node
  → [task_execution_node → approval_check_node? → progress_tracking_node → completion_check_node]*
  → report_generation_node
  → END
```

**Conditional Routing:**
- `approval_check_node`: Only if `requires_human_approval: true`
- `completion_check_node`: Routes back to `task_execution_node` if not complete

**Loop Pattern:**
- Task execution loop continues until `mission_status == "completed"`

### 4.4 Error Handling Strategy

**Error Collection:**
- All nodes append to `errors` list instead of raising exceptions
- Errors are non-fatal (workflow continues when possible)
- Final report includes error summary

**Error Types:**
1. **Data Errors:** Missing files, invalid JSON, missing IDs
2. **Validation Errors:** Circular dependencies, missing agents, invalid KPIs
3. **Execution Errors:** Agent execution failures, approval timeouts
4. **State Errors:** Missing required state fields

**Error Handling Pattern:**
```python
def some_node(state):
    errors = state.get("errors", [])
    try:
        # ... node logic ...
        return {"result": ..., "errors": errors}
    except Exception as e:
        return {"errors": errors + [f"some_node: {str(e)}"]}
```

### 4.5 Testing Strategy

**Unit Tests (Utilities First):**
1. Test data loading utilities with real JSON files
2. Test task ordering with various dependency patterns
3. Test agent selection logic
4. Test progress/KPI calculations

**Integration Tests (Nodes):**
1. Test goal_node → planning_node flow
2. Test data_loading_node with all data files
3. Test task_execution_node with mock agents
4. Test complete workflow end-to-end

**Test Data:**
- Use existing JSON files in `data/` directory
- Create test fixtures for edge cases (circular deps, missing agents, etc.)

---

## 5. Implementation Checklist

### Phase 0: Planning ✅
- [x] Deep data analysis
- [x] Decision rule mapping
- [x] State schema design
- [x] Architecture planning
- [x] Utility structure planning
- [x] Node structure planning
- [x] Error handling strategy
- [x] Testing strategy

### Next Steps
- [ ] Add `MissionOrchestratorState` to `config.py`
- [ ] Create `MissionOrchestratorConfig` dataclass
- [ ] Begin Phase 1: Goal and Planning nodes

---

## 6. Key Design Decisions

### 6.1 MVP Simplifications

1. **Agent Execution:** Mock execution (no real agent calls)
2. **Human Approval:** Simulated (no real UI, just state flag)
3. **Task Parallelization:** Sequential only (no parallel execution)
4. **Agent Selection:** First capable agent (no load balancing)
5. **Error Recovery:** Basic (log errors, continue when possible)

### 6.2 Future Enhancements (Post-MVP)

1. Real agent integration (API calls, function calls)
2. Real HITL workflows (approval UI, notifications)
3. Parallel task execution (when dependencies allow)
4. Agent load balancing and availability checking
5. Advanced error recovery and retry logic
6. LLM enhancement for task descriptions and reports

---

**End of Phase 0 Planning Document**



# Mission Orchestrator Agent

In [ ]:
# ============================================================================
# Mission Orchestrator Agent
# ============================================================================

class MissionOrchestratorState(TypedDict, total=False):
    """State for Mission Orchestrator Agent"""

    # Input fields
    mission_id: str                          # Mission to execute (e.g., "M001")

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                     # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Mission Data
    mission: Dict[str, Any]                 # Loaded mission object
    # Structure:
    # {
    #   "mission_id": "M001",
    #   "mission_name": "Reduce Customer Onboarding Time",
    #   "description": "..."
    # }

    mission_tasks: List[Dict[str, Any]]     # All tasks for the mission
    # Structure per task:
    # {
    #   "task_id": "T1",
    #   "task": "Collect customer information",
    #   "order": 1,
    #   "depends_on": [],
    #   "estimated_duration_minutes": 5,
    #   "requires_human_approval": false
    # }

    mission_kpis: Dict[str, Any]            # KPIs for the mission
    # Structure:
    # {
    #   "target_onboarding_time_days": 2,
    #   "baseline_onboarding_time_days": 5,
    #   "max_steps": 5
    # }

    # Agent Data
    specialized_agents: List[Dict[str, Any]] # All available agents
    # Structure per agent:
    # {
    #   "agent_id": "A1",
    #   "name": "Data Collection Agent",
    #   "description": "..."
    # }

    capabilities_matrix: List[Dict[str, Any]]  # Task-to-agent mappings
    # Structure per mapping:
    # {
    #   "task_id": "T1",
    #   "capable_agents": ["A1"]
    # }

    # Task Execution
    task_queue: List[Dict[str, Any]]        # Tasks ready to execute (dependencies satisfied)
    executed_tasks: List[Dict[str, Any]]    # Completed tasks with results
    # Structure per executed task:
    # {
    #   "task_id": "T1",
    #   "task": "...",
    #   "agent_id": "A1",
    #   "agent_name": "Data Collection Agent",
    #   "status": "completed" | "failed" | "awaiting_approval",
    #   "start_time": "2025-01-XX...",
    #   "end_time": "2025-01-XX...",
    #   "duration_minutes": 5.0,
    #   "result": {...},  # Agent execution result
    #   "error": Optional[str]
    # }

    current_task: Optional[Dict[str, Any]]   # Currently executing task
    task_results: Dict[str, Dict[str, Any]]  # task_id → execution result (for quick lookup)

    # Progress Tracking
    progress_percentage: float              # 0-100
    tasks_completed: int                    # Count of completed tasks
    tasks_total: int                        # Total tasks in mission
    elapsed_time_minutes: float             # Time since mission start
    estimated_remaining_minutes: float      # Estimated time to completion
    mission_start_time: Optional[str]        # ISO timestamp when mission started

    # KPI Metrics
    kpi_metrics: Dict[str, Any]              # Current KPI values
    # Structure:
    # {
    #   "actual_onboarding_time_days": 1.8,
    #   "actual_steps": 3,
    #   "improvement_percentage": 64.0  # (baseline - actual) / baseline * 100
    # }

    kpi_status: Dict[str, str]              # KPI achievement status
    # Structure:
    # {
    #   "onboarding_time": "on_track" | "at_risk" | "exceeded",
    #   "steps": "on_track" | "at_risk" | "exceeded"
    # }

    # HITL (Human-in-the-Loop)
    pending_approvals: List[Dict[str, Any]]  # Tasks awaiting human approval
    # Structure per approval:
    # {
    #   "task_id": "T2",
    #   "task": "Verify documents",
    #   "requested_at": "2025-01-XX...",
    #   "status": "pending" | "approved" | "rejected"
    # }

    approval_history: List[Dict[str, Any]]   # Approval decisions made
    # Structure per approval:
    # {
    #   "task_id": "T2",
    #   "decision": "approved" | "rejected",
    #   "decided_at": "2025-01-XX...",
    #   "decided_by": Optional[str]  # Human identifier (MVP: "human")
    # }

    # Mission Status
    mission_status: str                      # "not_started" | "in_progress" | "awaiting_approval" | "completed" | "failed"
    completion_reason: Optional[str]         # Why mission completed/failed

    # Output
    mission_report: str                      # Final markdown report
    report_file_path: Optional[str]         # Path to saved report file

    # Metadata
    errors: List[str]                       # Any errors encountered
    processing_time: Optional[float]      # Time taken to process


@dataclass
class MissionOrchestratorConfig:
    """Configuration for Mission Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/mission_reports"  # Where to save reports

    # Data file paths
    data_dir: str = "data"
    business_missions_file: str = "business_missions.json"
    decomposed_mission_tasks_file: str = "decomposed_mission_tasks.json"
    specialized_agents_file: str = "specialized_agents.json"
    agent_capabilities_matrix_file: str = "agent_capabilities_matrix.json"
    mission_kpis_file: str = "mission_kpis.json"

    # Task Execution Settings
    enable_parallel_execution: bool = False  # MVP: Sequential only
    task_timeout_minutes: int = 30           # Max time per task before timeout

    # HITL Settings
    approval_timeout_minutes: int = 60       # Max time to wait for approval
    auto_approve_for_testing: bool = False   # Auto-approve for testing (MVP)

    # Agent Selection
    agent_selection_strategy: str = "first_available"  # MVP: Simple selection
    # Future: "load_balanced", "skill_based", "cost_optimized"

    # Progress Tracking
    progress_update_interval_seconds: int = 5  # How often to update progress

    # KPI Tracking
    kpi_warning_threshold: float = 0.8      # Warn if KPI is 80% of target
    kpi_critical_threshold: float = 0.5     # Critical if KPI is 50% of target


# Nodes for Mission Orchestrator Agent

In [ ]:
"""Nodes for Mission Orchestrator Agent"""

from typing import Dict, Any
from config import MissionOrchestratorState


def goal_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for mission execution.

    This is a simple rule-based goal definition that sets the framework
    for mission orchestration.

    Input:
        - mission_id (str): Mission identifier

    Output:
        - goal (Dict): Goal definition
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    mission_id = state.get("mission_id")

    if not mission_id:
        return {
            "errors": errors + ["goal_node: mission_id is required"]
        }

    # Rule-based goal definition
    goal = {
        "objective": f"Execute mission {mission_id}",
        "mission_id": mission_id,
        "focus_areas": [
            "task_execution",
            "agent_coordination",
            "progress_tracking",
            "kpi_monitoring"
        ],
        "success_criteria": [
            "All tasks completed successfully",
            "KPIs within target thresholds",
            "Mission completed within estimated time"
        ]
    }

    return {
        "goal": goal,
        "errors": errors
    }


def planning_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan for mission execution.
    Rule-based, no LLM needed for MVP.

    Input:
        - goal (Dict): Goal definition from goal_node
        - mission_id (str): Mission identifier

    Output:
        - plan (List[Dict]): Execution plan
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    goal = state.get("goal")
    mission_id = state.get("mission_id")

    if not goal:
        return {
            "errors": errors + ["planning_node: goal is required"]
        }

    if not mission_id:
        return {
            "errors": errors + ["planning_node: mission_id is required"]
        }

    # Rule-based execution plan
    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load mission data, tasks, agents, and KPIs",
            "dependencies": [],
            "outputs": ["mission", "mission_tasks", "mission_kpis", "specialized_agents", "capabilities_matrix"]
        },
        {
            "step": 2,
            "name": "task_ordering",
            "description": "Order tasks and resolve dependencies",
            "dependencies": ["data_loading"],
            "outputs": ["task_queue"]
        },
        {
            "step": 3,
            "name": "task_execution",
            "description": "Execute tasks sequentially using specialized agents",
            "dependencies": ["task_ordering"],
            "outputs": ["executed_tasks", "task_results", "current_task"]
        },
        {
            "step": 4,
            "name": "approval_check",
            "description": "Handle human approval for tasks requiring HITL",
            "dependencies": ["task_execution"],
            "outputs": ["pending_approvals", "approval_history"],
            "conditional": True  # Only executes if tasks require approval
        },
        {
            "step": 5,
            "name": "progress_tracking",
            "description": "Calculate progress and KPI metrics",
            "dependencies": ["task_execution"],
            "outputs": ["progress_percentage", "kpi_metrics", "kpi_status"]
        },
        {
            "step": 6,
            "name": "completion_check",
            "description": "Check if mission is complete",
            "dependencies": ["progress_tracking"],
            "outputs": ["mission_status"],
            "conditional": True  # Routes based on completion status
        },
        {
            "step": 7,
            "name": "report_generation",
            "description": "Generate final mission report",
            "dependencies": ["completion_check"],
            "outputs": ["mission_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": errors
    }



# Tests for Mission Orchestrator nodes

In [ ]:
"""Tests for Mission Orchestrator nodes"""

import pytest
from agents.mission_orchestrator.nodes import goal_node, planning_node
from config import MissionOrchestratorState


def test_goal_node_success():
    """Test goal node with valid mission_id"""
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result
    assert result["goal"]["mission_id"] == "M001"
    assert result["goal"]["objective"] == "Execute mission M001"
    assert "focus_areas" in result["goal"]
    assert "success_criteria" in result["goal"]
    assert len(result.get("errors", [])) == 0


def test_goal_node_missing_mission_id():
    """Test goal node with missing mission_id"""
    state: MissionOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" not in result
    assert len(result.get("errors", [])) > 0
    assert "mission_id is required" in result["errors"][0]


def test_planning_node_success():
    """Test planning node with valid goal and mission_id"""
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "goal": {
            "objective": "Execute mission M001",
            "mission_id": "M001",
            "focus_areas": ["task_execution"],
            "success_criteria": ["All tasks completed"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result
    assert len(result["plan"]) > 0
    assert result["plan"][0]["name"] == "data_loading"
    assert result["plan"][0]["step"] == 1
    assert len(result.get("errors", [])) == 0


def test_planning_node_missing_goal():
    """Test planning node with missing goal"""
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    result = planning_node(state)

    assert "plan" not in result
    assert len(result.get("errors", [])) > 0
    assert "goal is required" in result["errors"][0]


def test_planning_node_missing_mission_id():
    """Test planning node with missing mission_id"""
    state: MissionOrchestratorState = {
        "goal": {
            "objective": "Execute mission M001",
            "mission_id": "M001"
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" not in result
    assert len(result.get("errors", [])) > 0
    assert "mission_id is required" in result["errors"][0]


def test_goal_and_planning_flow():
    """Test goal node followed by planning node"""
    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Execute goal node
    state = goal_node(state)
    assert "goal" in state
    assert len(state.get("errors", [])) == 0

    # Execute planning node
    state = planning_node(state)
    assert "plan" in state
    assert len(state.get("errors", [])) == 0
    assert len(state["plan"]) == 7  # Should have 7 steps
    assert state["plan"][0]["name"] == "data_loading"
    assert state["plan"][-1]["name"] == "report_generation"



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python3 -m pytest tests/test_mission_orchestrator_nodes.py -v
============================================================ test session starts ============================================================
platform darwin -- Python 3.13.7, pytest-9.0.2, pluggy-1.6.0 -- /Users/micahshull/Documents/AI_LangGraph/AI_AGENTS_000_MissionOrchestratorAgent/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/micahshull/Documents/AI_LangGraph/AI_AGENTS_000_MissionOrchestratorAgent
plugins: anyio-4.12.0, langsmith-0.4.59, asyncio-1.3.0, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 6 items

tests/test_mission_orchestrator_nodes.py::test_goal_node_success PASSED                                                               [ 16%]
tests/test_mission_orchestrator_nodes.py::test_goal_node_missing_mission_id PASSED                                                    [ 33%]
tests/test_mission_orchestrator_nodes.py::test_planning_node_success PASSED                                                           [ 50%]
tests/test_mission_orchestrator_nodes.py::test_planning_node_missing_goal PASSED                                                      [ 66%]
tests/test_mission_orchestrator_nodes.py::test_planning_node_missing_mission_id PASSED                                                [ 83%]
tests/test_mission_orchestrator_nodes.py::test_goal_and_planning_flow PASSED                                                          [100%]

============================================================= 6 passed in 0.02s =============================================================
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent %
